<a href="https://colab.research.google.com/github/hnipun/ColabProjects/blob/master/project_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#%%
import gzip
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D

IMAGE_SIZE = 28
TRAIN_IMAGES = 60000
TEST_IMAGES = 10000 
#%%

In [46]:
def show_image(image):
    image = np.asarray(image).squeeze()
    plt.imshow(image)
    plt.show()

def process_images(image_file, label_file, num_images):
    count = {}
    for i in range(0,11):
      count[i] = 0
    images = []
    labels = []
    
    
    f = gzip.open(image_file,'r')
    f.read(16)
    buf = f.read(IMAGE_SIZE * IMAGE_SIZE * num_images)
    data = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)
    data = data.reshape(num_images, IMAGE_SIZE, IMAGE_SIZE, 1)
    
    f = gzip.open(label_file,'r')
    f.read(8)
    for index in range(0, num_images):   
        buf = f.read(1)
        labels = np.frombuffer(buf, dtype=np.uint8).astype(np.int64)
        
        if not count[labels[0]] >= num_images//100:
          images.append(data[index])
          labels.append(labels[0])
          count[labels[0]]+=1
        else:
          continue
        
    return images,  labels
#%%
train_images, train_labels = process_images('/content/train-images-idx3-ubyte.gz', '/content/train-labels-idx1-ubyte.gz', TRAIN_IMAGES)
test_images, test_labels  = process_images('/content/t10k-images-idx3-ubyte.gz', '/content/t10k-labels-idx1-ubyte.gz', TEST_IMAGES)
#%%
show_image(train[0])
#%%

AttributeError: ignored

In [43]:
# create model
model = Sequential()

# add model layers
model.add(Conv2D(6, kernel_size=3, activation='relu', strides=(1, 1), input_shape=(28,28,1)))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))
model.add(Conv2D(16, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))
model.add(Flatten())
model.add(Dense(120, activation='relu'))
model.add(Dense(84, activation='relu'))
model.add(Dense(10, activation='softmax'))
#summary
model.summary()

#compile model using accuracy to measure model performance
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#train the model
model.fit(train['image'].values, train['label'].values, validation_data=(test['image'].values, test['label'].values), epochs=3)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 26, 26, 6)         60        
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 25, 25, 6)         0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 23, 23, 16)        880       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 22, 22, 16)        0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 7744)              0         
_________________________________________________________________
dense_10 (Dense)             (None, 120)               929400    
_________________________________________________________________
dense_11 (Dense)             (None, 84)               

ValueError: ignored

In [44]:
array = train['image'].values[1:3]
print(np.shape(train['image'].values))

(6000,)
